In [4]:
import inseq

In [74]:
from transformers import AutoModelWithLMHead, AutoTokenizer,AutoModelForCausalLM
import torch
def get_next_token_probabilities(model, input_embeddings):
    with torch.no_grad():
        outputs = model(inputs_embeds=input_embeddings)
        logits = outputs.logits[:, -1, :]  # Get the logits for the last token
        probabilities = torch.softmax(logits, dim=-1)
    return probabilities

## testing soft suff

In [92]:
model = AutoModelWithLMHead.from_pretrained("gpt2-medium")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

/home/cass/anaconda3/envs/eva/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [116]:
input_string = [ "I landed in Greece and I was staying in the capital city of" ]
input_ids = tokenizer(input_string, return_tensors='pt')['input_ids'].to(model.device)

### let s test 3 different importance distribution 1. flat 2. random 3. empirical correct
importance_scores = torch.softmax(torch.log(torch.tensor([[ 0, 0, 0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]], dtype=torch.float, device=model.device)), -1)
# and are removed heavily
strong_importance_scores = torch.softmax(torch.log(torch.tensor([[ 0, 0, 0, 0, 0.8, 0, 0, 0, 0, 0, 0.1, 0.1, 0]], dtype=torch.float, device=model.device)), -1)
# greece are removed heavily
correct_importance_scores = torch.softmax(torch.log(torch.tensor([[ 0, 0, 0, 0, 0, 0.8, 0, 0, 0, 0, 0.1, 0.1, 0]], dtype=torch.float, device=model.device)), -1)


input_wte = model.transformer.wte(input_ids)


In [117]:
next_token_probabilities = get_next_token_probabilities(model, input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)

print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities[:,next_token_id])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)
print(' ')

Next Token Probability Distribution over all probability:
tensor([[2.4524e-06, 2.8481e-06, 2.2075e-08,  ..., 1.6170e-09, 8.6779e-10,
         1.2370e-05]])
tensor([0.5257])
Next Token ID: tensor(21891)
Next Token:  Athens
 


In [118]:
embeddings_3rd = input_wte.size(2)
importance_scores = importance_scores.unsqueeze(2).repeat(1, 1, embeddings_3rd)
zeroout_mask = torch.bernoulli(1-importance_scores).to(model.device)
soft_input_wte = input_wte * zeroout_mask


next_token_probabilities = get_next_token_probabilities(model, soft_input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)

print("flat importance distribution, should be less change".center(50, "-"))
print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities[:,next_token_id])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)
print(' ')

flat importance distribution, should be less change
Next Token Probability Distribution over all probability:
tensor([[2.7316e-06, 3.8048e-06, 2.1670e-08,  ..., 7.7315e-10, 7.0193e-10,
         1.3444e-05]])
tensor([0.4927])
Next Token ID: tensor(21891)
Next Token:  Athens
 


In [119]:
embeddings_3rd = input_wte.size(2)
# using strong skewed importance score (incorrect one), unimportance ones are removed, important ones are keeeps, expect small changes
importance_scores = strong_importance_scores.unsqueeze(2).repeat(1, 1, embeddings_3rd)
zeroout_mask = torch.bernoulli(1-importance_scores).to(model.device)
soft_input_wte = input_wte * zeroout_mask


next_token_probabilities = get_next_token_probabilities(model, soft_input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)

print("skewed importance distribution, should lead to big changes".center(50, "-"))
print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities[:,next_token_id])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)
print(' ')

skewed importance distribution, should lead to big changes
Next Token Probability Distribution over all probability:
tensor([[4.7157e-06, 8.1449e-06, 2.7785e-08,  ..., 2.7829e-10, 2.2713e-10,
         2.4398e-05]])
tensor([0.2394])
Next Token ID: tensor(21891)
Next Token:  Athens
 


In [120]:
print("correct importance distribution, should lead to small changes".center(50, "-"))
embeddings_3rd = input_wte.size(2)
# using strong skewed importance score (incorrect one), unimportance ones are removed, important ones are keeeps, expect small changes
importance_scores = correct_importance_scores.unsqueeze(2).repeat(1, 1, embeddings_3rd)
zeroout_mask = torch.bernoulli(1-importance_scores).to(model.device)
soft_input_wte = input_wte * zeroout_mask


next_token_probabilities = get_next_token_probabilities(model, soft_input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)


print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities[:,next_token_id])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)
print(' ')

correct importance distribution, should lead to small changes
Next Token Probability Distribution over all probability:
tensor([[3.2545e-06, 3.6667e-06, 2.4394e-08,  ..., 1.1526e-09, 6.0725e-10,
         1.5459e-05]])
tensor([0.3688])
Next Token ID: tensor(21891)
Next Token:  Athens
 


In [84]:
# change "I" to "You", change "Greece" to "nowhere"
masked_input_string = [ "You landed in nowhere and I was staying in the capital city of" ]
masked_input_ids = tokenizer(masked_input_string, return_tensors='pt')['input_ids'].to(model.device)
masked_input_wte = model.transformer.wte(masked_input_ids)

next_token_probabilities = get_next_token_probabilities(model, masked_input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)

print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities.size())
print(next_token_probabilities[:,next_token_id])
print("the probability of 'Athens'", next_token_probabilities[:,21891])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)

Next Token Probability Distribution over all probability:
tensor([[2.0519e-05, 1.7326e-05, 2.0323e-07,  ..., 6.2910e-08, 5.7693e-08,
         2.1967e-05]])
torch.Size([1, 50257])
tensor([0.0964])
the probability of 'Athens' tensor([0.0026])
Next Token ID: tensor(262)
Next Token:  the


In [85]:
masked_input_string = [ "You landed in Greece and I was staying in the capital city of" ]
masked_input_ids = tokenizer(masked_input_string, return_tensors='pt')['input_ids'].to(model.device)
masked_input_wte = model.transformer.wte(masked_input_ids)

next_token_probabilities = get_next_token_probabilities(model, masked_input_wte)
next_token_id = torch.argmax(next_token_probabilities)
next_token = tokenizer.decode(next_token_id)

print("Next Token Probability Distribution over all probability:")
print(next_token_probabilities)
print(next_token_probabilities.size())
print(next_token_probabilities[:,next_token_id])
print("the probability of 'Athens'", next_token_probabilities[:,21891])
print("Next Token ID:", next_token_id)
print("Next Token:", next_token)



Next Token Probability Distribution over all probability:
tensor([[2.1874e-06, 1.7047e-06, 1.1743e-08,  ..., 1.7569e-09, 6.3192e-10,
         8.1974e-06]])
torch.Size([1, 50257])
tensor([0.8049])
the probability of 'Athens' tensor([0.8049])
Next Token ID: tensor(21891)
Next Token:  Athens


## testing package inseq

In [14]:
Model = "facebook/bart-base"
'''
"decapoda-research/llama-7b-hf"
"Helsinki-NLP/opus-mt-en-fr" 
"facebook/bart-base"  558M
'''

FA = "integrated_gradients"
'''
saliency input_x_gradient integrated_gradients deeplift gradient_shap discretized_integrated_gradients
attention
occlusion lime
'''

text = "when my flight landed in Greece, I converted my currency and slowly fall asleep. I was staying in the capital of"

In [26]:
model = inseq.load_model("gpt2", "input_x_gradient") 
out = model.attribute(text, generation_args={"max_new_tokens": 1}, n_steps=500, internal_batch_size=50, attribute_target=True,)
out.show()

attribute_target parameter is set to True, but will be ignored (not an encoder-decoder).
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with input_x_gradient...: 100%|██████████| 45/45 [00:00<00:00, 52.08it/s]


,ĠTokyo
When,0.023
Ġmy,0.017
Ġflight,0.028
Ġlanded,0.019
Ġin,0.034
ĠJapan,0.077
",",0.058
ĠI,0.008
Ġconverted,0.014
Ġmy,0.006


In [24]:
model = inseq.load_model("gpt2", "saliency") # saliency integrated_gradients
out = model.attribute(text, generation_args={"max_new_tokens": 1}, n_steps=500, internal_batch_size=50, attribute_target=True,)
out.show()

attribute_target parameter is set to True, but will be ignored (not an encoder-decoder).
/home/cass/anaconda3/envs/eva/lib/python3.11/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with saliency...: 100%|██████████| 45/45 [00:00<00:00, 49.57it/s]


,ĠTokyo
When,0.036
Ġmy,0.02
Ġflight,0.031
Ġlanded,0.02
Ġin,0.029
ĠJapan,0.081
",",0.028
ĠI,0.008
Ġconverted,0.014
Ġmy,0.008


In [27]:
model = inseq.load_model("facebook/bart-base", "saliency")
out = model.attribute(text, generation_args={"max_new_tokens": 1}, n_steps=500, internal_batch_size=50)
out.show()

Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with saliency...: 100%|██████████| 3/3 [00:00<00:00, 57.37it/s]


,<s>,</s>
<s>,0.007,0.012
When,0.01,0.126
Ġmy,0.006,0.022
Ġflight,0.013,0.028
Ġlanded,0.016,0.024
Ġin,0.007,0.016
ĠJapan,0.019,0.062
",",0.012,0.016
ĠI,0.012,0.017
Ġconverted,0.038,0.031


In [1]:

model = inseq.load_model("Helsinki-NLP/opus-mt-en-fr", "integrated_gradients")
out = model.attribute(
    "The developer argued with the designer because she did not like the design.",
    n_steps=300,
    return_convergence_delta=True,
    step_scores=["probability"],
)
out.show()

/home/cass/anaconda3/envs/eva/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-17 08:57:10.228590: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 08:57:10.443332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 08:57:11.271559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not 

,▁Le,▁développeur,▁s,',est,▁disputé,▁avec,▁la,▁designer,▁parce,▁qu,',elle,▁n,',aimait,▁pas,▁le,▁design,.,</s>
▁The,0.157,0.076,0.047,0.095,0.055,0.042,0.111,0.022,0.023,0.03,0.029,0.042,0.069,0.017,0.045,0.026,0.033,0.053,0.027,0.023,0.07
▁developer,0.312,0.415,0.099,0.086,0.093,0.09,0.079,0.099,0.075,0.099,0.037,0.084,0.059,0.026,0.057,0.043,0.03,0.077,0.105,0.036,0.072
▁argued,0.085,0.088,0.407,0.189,0.19,0.414,0.221,0.044,0.059,0.101,0.038,0.108,0.04,0.021,0.078,0.042,0.025,0.076,0.079,0.033,0.05
▁with,0.025,0.03,0.104,0.06,0.05,0.114,0.228,0.032,0.04,0.059,0.022,0.04,0.03,0.015,0.033,0.026,0.023,0.041,0.029,0.031,0.043
▁the,0.035,0.049,0.054,0.047,0.032,0.054,0.09,0.095,0.111,0.273,0.026,0.057,0.024,0.015,0.027,0.033,0.019,0.04,0.038,0.047,0.038
▁designer,0.046,0.072,0.05,0.059,0.036,0.054,0.045,0.293,0.413,0.104,0.062,0.093,0.038,0.029,0.057,0.039,0.038,0.138,0.118,0.072,0.061
▁because,0.038,0.025,0.041,0.053,0.055,0.036,0.035,0.054,0.043,0.125,0.114,0.092,0.073,0.037,0.046,0.045,0.075,0.029,0.028,0.026,0.06
▁she,0.142,0.056,0.034,0.071,0.059,0.038,0.054,0.186,0.073,0.043,0.289,0.084,0.385,0.045,0.063,0.046,0.047,0.039,0.051,0.039,0.068
▁did,0.036,0.031,0.027,0.052,0.211,0.024,0.027,0.035,0.028,0.033,0.182,0.12,0.112,0.163,0.14,0.179,0.083,0.027,0.044,0.028,0.042
▁not,0.017,0.019,0.026,0.041,0.11,0.025,0.022,0.023,0.016,0.024,0.074,0.059,0.048,0.388,0.186,0.141,0.234,0.026,0.034,0.033,0.056
